In [32]:
import random
from tqdm import tqdm
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from binascii import unhexlify

# Define Parameter

In [1]:
F = GF(2**3, 'x')
G = AlternatingGroup(5)
group_elements = list(G)
field_elements = list(F)
n = G.order()
A = GroupAlgebra(G, F)
M = MatrixSpace(A, 3,3)

In [3]:
id_mat = M([[A.monomial(group_elements[0]), A.zero(), A.zero()],
            [A.zero(), A.monomial(group_elements[0]), A.zero()],
            [A.zero(), A.zero(), A.monomial(group_elements[0])]])
id_mat

[()  0  0]
[ 0 ()  0]
[ 0  0 ()]

# Define Matrix Sampling Function

In [4]:
def is_idempotent(A):
    return A * A == A

In [ ]:
x = F.gen()

# Here we define a non invertible element in the group algebra A
# This is later used to create a non-invertible matrix
s = (
    x^2             * A.monomial(G((3,4,5))) +
    (x^2 + x)       * A.monomial(G((2,5,4))) +
    (x^2 + x + 1)   * A.monomial(G((1,3,5,2,4))) +
    x               * A.monomial(G([(1,3), (2,5)])) +
    (x + 1)         * A.monomial(G((1,4,3,5,2))) +
    (x^2 + x + 1)   * A.monomial(G([(1,4), (2,3)])) +
    x               * A.monomial(G((1,5,4,2,3))) +
    1               * A.monomial(G((1,5,2,3,4)))
)

In [31]:
try:
    s_inv = s.inverse()
    print("The element s is invertible")
except ValueError:
    print("The element s is not invertibile")

The element s is not invertibile


In [ ]:
def random_algebra_element():

    # Since I'm not sure about the random_element() method in SageMath, so I write it manually.
    a = A.zero()
    for i in range(n):
        coeff = random.choice(field_elements)
        a += coeff * A.monomial(group_elements[i])
    return a

def random_invertible_triangular_matrix(pos = "upper"):
    mat = [[0,0,0],[0,0,0],[0,0,0]]

    # The triangular matrix is in the form of:
    # U = [[g1, a1, a2],
    #      [0, g2, a3],
    #      [0, 0, g3]]
    # where g1, g2, g3 are elements of group G and a1, a2, a3 are arbitrary elements of algebra A.

    if pos == "upper":
        for i in range(3):
            for j in range(i, 3):
                if i == j:
                    mat[i][j] = A.monomial(random.choice(group_elements)) 
                else:
                    mat[i][j] = random_algebra_element()
        return M(mat)
    
    elif pos == "lower":
        for i in range(3):
            for j in range(i + 1):
                if i == j:
                    mat[i][j] = A.monomial(random.choice(group_elements)) 
                else:
                    mat[i][j] = random_algebra_element()
        return M(mat)
    
    else:
        raise TypeError("pos has to be either upper or lower")
        

def random_invertible_matrix():

    # The sampling method for invertible matrices is by multiplying random invertible triangular matrices by factor of 20.
    mat = id_mat
    for i in range(20):
        pos = random.choice(["upper", "lower"])
        mat *= random_invertible_triangular_matrix(pos)
    return mat
        

def random_non_invertible_matrix():

    # The sampling method for non-invertible matrices is simply by multiplying a random invertible matrix by a fixed non-inverible scalar s.
    return s*random_invertible_matrix()        

In [9]:
mat_1 = random_invertible_matrix()
mat_1

[                                                                                                              (3,4,5) + (2,3)(4,5) + x*(2,3,4) + (x^2+x)*(2,3,5) + (x^2+x)*(2,4,3) + (x^2+x+1)*(2,4)(3,5) + (x+1)*(2,5,3) + (x^2+x+1)*(2,5,4) + (x^2+1)*(1,2)(4,5) + (x^2+1)*(1,2)(3,4) + (1,2)(3,5) + (x^2+x+1)*(1,2,3) + x*(1,2,3,4,5) + (1,2,3,5,4) + (x^2+1)*(1,2,4,5,3) + (x^2+x)*(1,2,4) + (x^2+x)*(1,2,4,3,5) + (x^2+x+1)*(1,2,5,4,3) + x^2*(1,3,2) + (x+1)*(1,3,4,5,2) + (1,3,5,4,2) + (x+1)*(1,3,4) + x^2*(1,3,5) + x^2*(1,3)(2,4) + (x+1)*(1,3,2,4,5) + x*(1,3,5,2,4) + (x^2+x)*(1,3)(2,5) + x^2*(1,3,2,5,4) + x*(1,3,4,2,5) + (x+1)*(1,4,2) + (x^2+x+1)*(1,4,3,5,2) + (x^2+x+1)*(1,4,3) + x^2*(1,4,5) + (x^2+1)*(1,4)(3,5) + x^2*(1,4)(2,3) + (1,4,2,3,5) + (x+1)*(1,4,2,5,3) + (x+1)*(1,4,3,2,5) + (x+1)*(1,4)(2,5) + x*(1,5,4,3,2) + (x^2+1)*(1,5,2) + x^2*(1,5,3,4,2) + (x+1)*(1,5,4) + (x+1)*(1,5)(3,4) + (x+1)*(1,5,4,2,3) + (1,5)(2,3) + (x^2+x)*(1,5,2,3,4) + (x^2+1)*(1,5,2,4,3) + (1,5,3,2,4) + (x^2+x+1)*(1,5)(2,4

In [10]:
mat_s = random_non_invertible_matrix()
mat_s

[   (x+1)*() + (x+1)*(3,4,5) + (x^2+x+1)*(3,5,4) + x^2*(2,3)(4,5) + (x^2+x+1)*(2,3,4) + (x^2+x+1)*(2,3,5) + (x^2+x)*(2,4,3) + x^2*(2,4,5) + (x^2+1)*(2,4)(3,5) + (x^2+x)*(2,5,3) + (x^2+x+1)*(2,5,4) + (x+1)*(2,5)(3,4) + x*(1,2)(4,5) + (x^2+x+1)*(1,2)(3,4) + (x^2+1)*(1,2)(3,5) + (1,2,3) + (x+1)*(1,2,3,4,5) + (1,2,3,5,4) + (x^2+x+1)*(1,2,4,5,3) + x^2*(1,2,4) + (x^2+1)*(1,2,4,3,5) + (1,2,5,4,3) + x*(1,2,5) + (1,3,2) + (x^2+x)*(1,3,4,5,2) + x^2*(1,3,5,4,2) + (1,3)(4,5) + (x^2+x+1)*(1,3,4) + (x^2+x+1)*(1,3,5) + x*(1,3)(2,4) + (x^2+x)*(1,3,2,4,5) + (1,3,5,2,4) + (x+1)*(1,3)(2,5) + (x^2+x)*(1,3,2,5,4) + (x^2+x)*(1,3,4,2,5) + (x+1)*(1,4,5,3,2) + (x^2+x)*(1,4,2) + (x^2+x)*(1,4,3,5,2) + x^2*(1,4,3) + (x^2+x+1)*(1,4,5) + (1,4)(3,5) + (x^2+x)*(1,4,5,2,3) + (x+1)*(1,4)(2,3) + x*(1,4,2,3,5) + (x^2+x)*(1,4,2,5,3) + (x^2+1)*(1,4,3,2,5) + (x^2+x)*(1,4)(2,5) + (x^2+1)*(1,5,4,3,2) + (1,5,2) + x^2*(1,5,3,4,2) + (1,5,3) + (x^2+x)*(1,5,4) + (x^2+x+1)*(1,5)(3,4) + x*(1,5,4,2,3) + (x+1)*(1,5,2,3,4) + x*(1,5,2,4

# Key Exchange and AES

In [ ]:
def poly_to_bitstring(elem):
    
    # Here, we convert an element of the field F to a bitstring representation.
    poly = elem.polynomial()
    coeffs = poly.coefficients(sparse=False)
    coeffs += [0] * (3 - len(coeffs)) 
    return ''.join(str(int(b)) for b in reversed(coeffs))

In [14]:
y = random_non_invertible_matrix()
while is_idempotent(y):
    y = random_non_invertible_matrix()
g = random_non_invertible_matrix()

In [29]:
matrix_key = y*(g**(random.randint(1, 2**100)))
first_entry = matrix_key[0,0]
bitstring_entry = ''.join(poly_to_bitstring(first_entry[g]) for g in group_elements)
shared_key = bitstring_entry[:128]
print(len(shared_key))
shared_key = int(shared_key, 2).to_bytes(16, 'big')
print(shared_key)

128
b'\xef\xd8^\xe2X\x04\xf3T\xfe<O\x81\x16JD\xef'


In [27]:
def key_exchange():
    
    # This function performs the key exchange process using non-invertible matrices.
    y = random_non_invertible_matrix()
    while is_idempotent(y):
        y = random_non_invertible_matrix()
    g = random_non_invertible_matrix()

    a_secret = random.randint(1, 2**100)
    b_secret = random.randint(1, 2**100)
    a_key = y*(g**a_secret)
    b_key = y*(g**b_secret)
    matrix_key = a_key * (g**b_secret)
    
    # The first entry of the matrix_key is used to derive the shared key.
    first_entry = matrix_key[0,0]
    bitstring_entry = ''.join(poly_to_bitstring(first_entry[g]) for g in group_elements)
    shared_key = bitstring_entry[:128]
    shared_key = int(shared_key, 2).to_bytes(16, 'big')
    return shared_key, a_key, b_key

In [28]:
shared_key = key_exchange()[0]

iv = get_random_bytes(16)

cipher_encrypt = AES.new(shared_key, AES.MODE_CBC, iv)

plaintext = b'Super Secret Message!!!'

padded = pad(plaintext, AES.block_size)
ciphertext = cipher_encrypt.encrypt(padded)

print("IV (hex):", iv.hex())
print("Ciphertext (hex):", ciphertext.hex())

cipher_decrypt = AES.new(shared_key, AES.MODE_CBC, iv)
decrypted_padded = cipher_decrypt.decrypt(ciphertext)
decrypted = unpad(decrypted_padded, AES.block_size)

print("Decrypted text:", decrypted)

IV (hex): c253b297e9ec5ebfb8dba9af57ed451b
Ciphertext (hex): 62aa0ec03c2f9e9abeb4e9bb10b525548b41f17068a31175afd2777d158ebff3
Decrypted text: b'Super Secret Message!!!'


# An attempt to find how long the orbit could take

In [30]:
def generate_shifted_orbit(y,g):
    shifted_g = {}
    shifted_g['Elements'] = [y*g]
    element = y*(g*g)
    size = 1
    pbar = tqdm(desc = "Generating shifted orbit", unit = 'iter')
    while element not in shifted_g['Elements']:
        shifted_g['Elements'].append(element)
        element *= g
        size += 1
        pbar.update(1)
    index = shifted_g['Elements'].index(element)
    cycle_length = size - index
    shifted_g['Size'] = size
    shifted_g['Index'] = index
    shifted_g['CycleLength'] = cycle_length
    return shifted_g

In [18]:
generate_shifted_orbit(y,g)

Generating shifted orbit: 199242iter [9:00:00, 32.61s/iter] 

: 